### Basic RAG retriever using Langchain and Anthropic model

In [1]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_anthropic import ChatAnthropic
from langchain_community.document_loaders.csv_loader import CSVLoader
from dotenv import load_dotenv
import os
load_dotenv()
os.environ["LANGCHAIN_API_KEY"]= os.getenv("LANGCHAIN_API_KEY")
llm = ChatAnthropic(model="claude-3-5-sonnet-20240620")

loader= CSVLoader(file_path='../notebooks/data/cleaned20242108.csv',source_column='nombre')
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100
                                               )
splits = text_splitter.split_documents(docs)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
first= splits[0:5000]
second= splits[5000:10000]
third= splits[10000:15000]
fourth= splits[15000:]

In [4]:
persist_directory='db'

In [6]:
vectorstore = Chroma.from_documents(persist_directory=persist_directory,documents=fourth, embedding=OllamaEmbeddings(model='nomic-embed-text'))

In [13]:
# vectorestore= Chroma(persist_directory=persist_directory,embedding_function=OllamaEmbeddings(model='nomic-embed-text'),)

In [8]:
# query="cual es la cervezeria con mejor puntuacion del puente de vallecas?"

In [12]:

retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("Mis amigos y yo estaremos haciendo un restaurant crawl en Malasaña \
                y solo tenemos 100 euros, queremos ir a 3 lugares minimo y tapear, \
                    cuales restaurantes o bares me recomiendas?")